## NeighLoR notebook
In this notebook, we evaluate the following models:
1. NeighLoR
2. Logistic regression
3. NeighLoR + pretrained Word2Vec
4. NeighLoR + custom Word2Vec
5. Logistic regression + pretrained Word2Vec
6. Logistic regression + custom Word2Vec

In [20]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score
from scipy.special import logit, expit
import pickle
from multiprocessing import Pool
from copy import deepcopy
from sklearn.linear_model import LogisticRegression

In [47]:
"""Loading the representations and the data"""

data = pickle.load(open('./processed_BA.b','rb'))
actual_words_5, actual_words_7, actual_words_9 = pickle.load(open('./neighborhoods_ba.b','rb'))
full_representation = pickle.load(open('./full_ba.b','rb'))
lem_verbs, lem_adj, lem_adv, lem_words, unique_verbs, unique_adverbs, unique_adjectives = pickle.load(open('./vocab_BA.b','rb'))
from_ba = pickle.load(open('./from_ba.b','rb'))
data['from_ba'] = from_ba

In [10]:
"""Feature extraction"""
ys = list(data.pi)
verb_cond = {}
adverb_cond = {}
adj_cond = {}
verb_count = {}
adverb_count = {}
adj_count = {}

unique_verb = list(lem_verbs.keys())
unique_adv = list(set(list(lem_adv)).difference(unique_verbs))
unique_adj = list(set(lem_adj).difference(unique_adverbs).difference(unique_verbs))

for k in tqdm(range(data.shape[0])):
    """Loop over the tweets"""
    y = ys[k]
    local_pattern = full_representation[k]
    visited = set()
    for w in local_pattern:
        if w.count('-')<=1 and w.count('/')<=1 and not(any(str(k) in w for k in range(10))) and not('#' in w):
            w = w.replace('-',' ').replace('/',' ').replace('\\','')
            if w in unique_verb:
                try:
                    verb_count[lem_verbs[w]] += 1
                    verb_cond[lem_verbs[w]] += data.pi.iloc[k]
                except:
                    verb_count[lem_verbs[w]] = 1
                    verb_cond[lem_verbs[w]] = data.pi.iloc[k]
            elif w in unique_adv:
                try:
                    adverb_count[w] += 1
                    adverb_cond[w] += data.pi.iloc[k]
                except:
                    adverb_count[w] = 1
                    adverb_cond[w] = data.pi.iloc[k]
            elif w in unique_adj:
                try:
                    adj_count[lem_adj[w]] += 1
                    adj_cond[lem_adj[w]] += data.pi.iloc[k]
                except:
                    adj_count[lem_adj[w]] = 1
                    adj_cond[lem_adj[w]] = data.pi.iloc[k]

In [11]:
"""Computing the probabilities"""
verb_cond = pd.Series(verb_cond)
adverb_cond = pd.Series(adverb_cond)
adj_cond = pd.Series(adj_cond)
verb_count = pd.Series(verb_count)
adverb_count = pd.Series(adverb_count)
adj_count = pd.Series(adj_count)

verb_cond = verb_cond/verb_count
adverb_cond = adverb_cond/adverb_count
adj_cond = adj_cond/adj_count

verb_cond.sort_values(ascending=False, inplace=True)
adverb_cond.sort_values(ascending=False, inplace=True)
adj_cond.sort_values(ascending=False, inplace=True)

verb_cond = pd.DataFrame([verb_cond, verb_count], index=['cond_proba','count']).T
adverb_cond = pd.DataFrame([adverb_cond, adverb_count], index=['cond_proba','count']).T
adj_cond = pd.DataFrame([adj_cond, adj_count], index=['cond_proba','count']).T

verb_cond['lower_bound'] = (verb_cond['cond_proba'] - 1.96*np.sqrt(verb_cond['cond_proba']*(1-verb_cond['cond_proba'])/verb_cond['count'])).apply(lambda x: max(x,0))
adverb_cond['lower_bound'] = (adverb_cond['cond_proba'] - 1.96*np.sqrt(adverb_cond['cond_proba']*(1-adverb_cond['cond_proba'])/adverb_cond['count'])).apply(lambda x: max(x,0))
adj_cond['lower_bound'] = (adj_cond['cond_proba'] - 1.96*np.sqrt(adj_cond['cond_proba']*(1-adj_cond['cond_proba'])/adj_cond['count'])).apply(lambda x: max(x,0))

verb_cond['cond_proba_no_pi'] = 1. - verb_cond['cond_proba']
verb_cond['lower_bound_no_pi'] = verb_cond['cond_proba_no_pi'] - 1.96 * np.sqrt(verb_cond['cond_proba_no_pi'] * (1-verb_cond['cond_proba_no_pi'])/verb_cond['count'])

adverb_cond['cond_proba_no_pi'] = 1. - adverb_cond['cond_proba']
adverb_cond['lower_bound_no_pi'] = adverb_cond['cond_proba_no_pi'] - 1.96 * np.sqrt(adverb_cond['cond_proba_no_pi'] * (1-adverb_cond['cond_proba_no_pi'])/adverb_cond['count'])

adj_cond['cond_proba_no_pi'] = 1. - adj_cond['cond_proba']
adj_cond['lower_bound_no_pi'] = adj_cond['cond_proba_no_pi'] - 1.96 * np.sqrt(adj_cond['cond_proba_no_pi'] * (1-adj_cond['cond_proba_no_pi'])/adj_cond['count'])

verb_cond = verb_cond[verb_cond['count']>=10]
adverb_cond = adverb_cond[adverb_cond['count']>=10]
adj_cond = adj_cond[adj_cond['count']>=10]

verb_features_10 = list(verb_cond.sort_values('lower_bound', ascending=False).index[:verb_cond.shape[0]//10]) + list(verb_cond.sort_values('lower_bound_no_pi', ascending=False).index[:verb_cond.shape[0]//10])
verb_features_20 = list(verb_cond.sort_values('lower_bound', ascending=False).index[:verb_cond.shape[0] * 2//10]) + list(verb_cond.sort_values('lower_bound_no_pi', ascending=False).index[:verb_cond.shape[0] * 2//10])
verb_features_30 = list(verb_cond.sort_values('lower_bound', ascending=False).index[:verb_cond.shape[0] * 3//10]) + list(verb_cond.sort_values('lower_bound_no_pi', ascending=False).index[:verb_cond.shape[0] * 3//10])

adverb_features_10 = list(adverb_cond.sort_values('lower_bound', ascending=False).index[:adverb_cond.shape[0]//10]) + list(adverb_cond.sort_values('lower_bound_no_pi', ascending=False).index[:adverb_cond.shape[0]//10])
adverb_features_20 = list(adverb_cond.sort_values('lower_bound', ascending=False).index[:adverb_cond.shape[0] * 2//10]) + list(adverb_cond.sort_values('lower_bound_no_pi', ascending=False).index[:adverb_cond.shape[0] * 2//10])
adverb_features_30 = list(adverb_cond.sort_values('lower_bound', ascending=False).index[:adverb_cond.shape[0] * 3//10]) + list(adverb_cond.sort_values('lower_bound_no_pi', ascending=False).index[:adverb_cond.shape[0] * 3//10])

adj_features_10 = list(adj_cond.sort_values('lower_bound', ascending=False).index[:adj_cond.shape[0]//10]) + list(adj_cond.sort_values('lower_bound_no_pi', ascending=False).index[:adj_cond.shape[0]//10])
adj_features_20 = list(adj_cond.sort_values('lower_bound', ascending=False).index[:adj_cond.shape[0] * 2//10]) + list(adj_cond.sort_values('lower_bound_no_pi', ascending=False).index[:adj_cond.shape[0] * 2//10])
adj_features_30 = list(adj_cond.sort_values('lower_bound', ascending=False).index[:adj_cond.shape[0] * 3//10]) + list(adj_cond.sort_values('lower_bound_no_pi', ascending=False).index[:adj_cond.shape[0] * 3//10])

"""Function to compute the one-hot encodings"""
features_10 = verb_features_10 + adverb_features_10 + adj_features_10
features_20 = verb_features_20 + adverb_features_20 + adj_features_20
features_30 = verb_features_30 + adverb_features_30 + adj_features_30

In [12]:
class MaxLog:
    
    def __init__(self, d, fit_intercept=True, init_w=None, init_b=None, alpha=1e-3, epsilon=1e-2):
        """Defining the dimension and fit_intercept parameters"""
        self.dim = d
        self.grad = np.zeros(self.dim)
        self.fit_intercept = fit_intercept
        
        """Initializing the weights"""
        self.w = 0.
        if init_w is None:
            self.w = np.random.normal(size=self.dim)
        elif isinstance(init_w, np.ndarray) and init_w.shape[0]==self.dim:
            self.w = init_w
        else:
            raise ValueError('Wrong dimension in intial parameters.')
        
        """Initializing the bias"""
        self.b = 0.
        if fit_intercept:
            if init_b is None:
                self.b = np.random.normal()
            elif isinstance(init_b, (float, np.float64)):
                self.b = init_b
            else:
                raise ValueError('Bias initialization must be a float type.')
                
        """Setting the learning rate"""
        if isinstance(alpha, (float, np.float64)):
            self.alpha = alpha
        else:
            raise ValueError('Learning rate must be a float type.')
        
        """Setting the stopping criterion for the gradient descent"""
        if isinstance(epsilon, (float, np.float64)):
            self.epsilon = epsilon
        else:
            raise ValueError('Tolerance must be a float type.')
        
        """Initializing error sequence"""
        self.errors = []
        
        pass
    
    
    def check_dim(self, X):
        """Computing check on dimension of observations"""
        dim_check = any(x.shape[1]!=self.dim for x in X)
        if dim_check:
            raise ValueError('Wrong dimension in observations')
        pass
    
    
    def check_attr(self, attr, message=''):
        """Function to check whether the instance has an attribute"""
        if not hasattr(self, attr):
            if message=='':
                raise ValueError("Instance doesn't have attribute '{}'".format(attr))
            else:
                raise ValueError(message)
        pass
    
    
    def predict(self, X):
        """Computing check on dimension of observations"""
        self.check_dim(X)
            
        """Computing the predictions"""
        pred = [np.max(expit(self.b + x.dot(self.w))) for x in X]
        return np.array([int(p>=0.5) for p in pred])
    
    
    def predict_proba(self, X):
        """Computing check on dimension of observations"""
        self.check_dim(X)
            
        """Computing the probability"""
        pred = [np.max(expit(self.b + x.dot(self.w))) for x in X]
        return np.array(pred)
    
    
    def fit(self, X, y, parallel=False, n_iter=None):
        """Function to fit the model
        Needs : compute the forward propagation, compute the backward propagation"""
        """Computing check on dimension of observations"""
        self.check_dim(X)
        
        """Initiating the error sequence"""
        self.errors = []
        self.grad = 10
        
        """Performing the gradient descent"""
        error = np.inf
        if n_iter==None:
            while error >= self.epsilon:
                self.forward(X, y)
                self.backward(X, y)
                error = self.grad
        else:
            for k in tqdm(range(n_iter), desc='Fitting the model'):
                self.forward(X, y)
                self.backward(X, y)
                if self.grad<=self.epsilon:
                    break
        pass
    
    
    def forward(self, X, y):
        """Performing the forward pass of the model : registering the maximum indexes, the current proba scores, and the error"""
        probas = [expit(self.b + x.dot(self.w)) for x in X]
        self.max_ind = [np.argmax(v) for v in probas]
        y_pred = np.array([np.max(v) for v in probas])
        self.eta = y_pred
        self.errors.append(- np.mean(np.array(y) * np.log(y_pred) + (1-np.array(y)) * np.log(1-y_pred)))
        pass
    
    
    def backward(self, X, y):
        grad_w, grad_b = self.gradient(X, y)
        self.w = self.w - self.alpha * grad_w
        self.b = self.b - self.alpha * grad_b
        self.grad = np.sqrt(np.sum(grad_w**2)+grad_b**2)
        pass
    
    
    def gradient(self, X, y):
        """Check on error and maximum index for gradient computation"""
        self.check_attr('errors', message='Need to compute forward pass to have error')
        self.check_attr('max_ind', message='Need to compute forward pass to have maximum index')
        self.check_attr('eta', message='Need to compute forward pass to have current estimate probabilities')
        
        """Computing the gradient for every observation"""
        mi = self.max_ind
        current_X = np.vstack([X[k][mi[k]].reshape((1,-1)) for k in range(len(X))])
        grad_w = current_X.T.dot(self.eta - np.array(y))/current_X.shape[0]
        grad_b = 0.
        if self.fit_intercept:
            grad_b = np.mean(self.eta - np.array(y))
        return grad_w, grad_b

In [13]:
"""Building representations"""
d = len(features_20)

def represent(x):
    return [int(w in x) for w in features_20]

X_basic = [[represent(pattern) for pattern in tweet] for tweet in tqdm(actual_words_7)]
X_basic = [np.array(tweet) for tweet in tqdm(X_basic)]

for k in tqdm(range(len(X_basic))):
    if X_basic[k].shape[0]==0:
        X_basic[k] = np.zeros((1,d))
        
X_basic = [np.hstack([X_basic[k], data.from_ba.iloc[k] * np.ones((X_basic[k].shape[0],1))]) for k in tqdm(range(data.shape[0]))]
features_20 = features_20 + ['from_ba']

In [17]:
"""Collecting the negative bigrams"""
import spacy

nlp = spacy.load('en')

def get_neg_pairs(text):
    doc = nlp(text)
    dep_df = pd.DataFrame([[token.text, token.pos_, token.dep_, token.head.text, token.head.pos_] for token in doc], columns=['token','token_pos','dep','head','head_pos'])
    neg_dep = dep_df[(dep_df.dep=='neg') & (dep_df.head_pos=='VERB')]
    
    from nltk import WordNetLemmatizer
    lmtzr = WordNetLemmatizer()
    pairs = [[neg_dep.token.iloc[i], lmtzr.lemmatize(neg_dep['head'].iloc[i], 'v')] for i in range(neg_dep.shape[0])]
    return pairs

p = Pool(40)

neg_pairs_data = []
N = data.shape[0]

for k in tqdm(range(N//500+1)):
    neg_pairs_data += p.map(get_neg_pairs, list(data.text.iloc[k*500:min(N,(k+1)*500)]))

Process ForkPoolWorker-60:
Process ForkPoolWorker-52:
Process ForkPoolWorker-69:
Process ForkPoolWorker-48:
Process ForkPoolWorker-75:
Process ForkPoolWorker-53:
Process ForkPoolWorker-80:
Process ForkPoolWorker-59:
Process ForkPoolWorker-62:
Process ForkPoolWorker-41:
Traceback (most recent call last):
Process ForkPoolWorker-56:
Process ForkPoolWorker-54:
Process ForkPoolWorker-68:
Process ForkPoolWorker-57:
Process ForkPoolWorker-65:
Process ForkPoolWorker-76:
Process ForkPoolWorker-78:
Process ForkPoolWorker-61:
Process ForkPoolWorker-64:
Process ForkPoolWorker-47:
Process ForkPoolWorker-42:
Process ForkPoolWorker-49:
Traceback (most recent call last):
Process ForkPoolWorker-43:
Process ForkPoolWorker-44:
Process ForkPoolWorker-51:
Traceback (most recent call last):
Process ForkPoolWorker-79:
Process ForkPoolWorker-45:
Traceback (most recent call last):
  File "/home/qrg-researchlab/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Process F

  File "/home/qrg-researchlab/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/qrg-researchlab/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/qrg-researchlab/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/qrg-researchlab/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/qrg-researchlab/anaconda3/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/qrg-researchlab/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/qrg-researchlab/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/qrg-researchlab/anacon

  File "/home/qrg-researchlab/anaconda3/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/home/qrg-researchlab/anaconda3/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/qrg-researchlab/anaconda3/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/home/qrg-researchlab/anaconda3/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/home/qrg-researchlab/anaconda3/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/qrg-researchlab/anaconda3/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
Process ForkPoolWorker-72:
  File "/home/qrg-researchlab/anaconda3/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
Process ForkPoolWorker-73:
  File "/home/qrg-researchlab/anaconda3/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._r

  File "/home/qrg-researchlab/anaconda3/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/home/qrg-researchlab/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/qrg-researchlab/anaconda3/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
  File "/home/qrg-researchlab/anaconda3/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
Process ForkPoolWorker-71:
  File "/home/qrg-researchlab/anaconda3/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
  File "/home/qrg-researchlab/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Keybo

In [18]:
"""Recording the verb bigrams"""
verb_bigrams = {}
for k in range(len(neg_pairs_data)):
    for i in range(len(neg_pairs_data[k])):
        if neg_pairs_data[k][i][1] in verb_features_30:
            try:
                verb_bigrams[neg_pairs_data[k][i][1]] += 1
            except:
                verb_bigrams[neg_pairs_data[k][i][1]] = 1

verb_bigrams = pd.Series(verb_bigrams)
verb_bigrams = verb_bigrams[verb_bigrams>=5]
verb_bigrams = list(verb_bigrams.index)

print("Number of verb bigrams : {}".format(len(verb_bigrams)))

Number of verb bigrams : 110


In [21]:
"""Building the negative bigrams representations for neighborhoods"""
neg_reps = []
neg_presence = []
for k in tqdm(range(data.shape[0])):
    nr = []
    npr = []
    
    for i in range(len(actual_words_7[k])):
        local = []
        neighborhood = set(actual_words_7[k][i])
        
        for neg_pair in neg_pairs_data[k]:
            if len(neighborhood.intersection(neg_pair))==2:
                if neg_pair[1] in verb_bigrams:
                    local.append(neg_pair[1])
                if neg_pair[0] in verb_bigrams:
                    local.append(neg_pair[0])
        
        if len(local)>0:
            npr.append(1)
        else:
            npr.append(0)
        nr.append(deepcopy(local))
    
    neg_reps.append(deepcopy(nr))
    neg_presence.append(np.array(npr).reshape((len(actual_words_7[k]),1)))
    
neg_reps = [np.array([[int(v in local) for v in verb_bigrams] for local in tweet]) for tweet in neg_reps]

"""Correcting the defects of negation representations"""
d_neg = len(verb_bigrams)

for k in range(len(neg_reps)):
    if neg_reps[k].shape[0]==0:
        neg_reps[k] = np.zeros((1,d_neg))
    if neg_presence[k].shape[0]==0:
        neg_presence[k] = np.zeros((1,1))

In [22]:
"""Complementing the representations for both methods of handling negations"""
X_neg = [np.hstack([X_basic[k], neg_reps[k]]) for k in range(len(X_basic))]
d = len(features_20) + len(verb_bigrams)
feat = features_20 + ['not_' + w for w in verb_bigrams]

## Evaluating the NeighLoR approach

In [24]:
"""Building indexes"""
n = len(ys)
d = X_neg[0].shape[1]

"""Retrieving the positive and negative indexes"""
neg_index = [k for k in range(len(ys)) if ys[k]==0]
pos_index = [k for k in range(len(ys)) if ys[k]==1]

"""Shuffling both the positive and negative indexes"""
np.random.seed(seed=0)
np.random.shuffle(neg_index)
np.random.shuffle(pos_index)

"""Computing train and test index sets for the chosen number of folds"""
cv = 10

s = len(neg_index)//cv
neg_index_list = [neg_index[k*s:(k+1)*s] for k in range(cv-1)]
neg_index_list.append(neg_index[(cv-1)*s:])
assert(np.sum([len(e) for e in neg_index_list])==len(neg_index)) # Check on negative index completion

s = len(pos_index)//cv
pos_index_list = [pos_index[k*s:(k+1)*s] for k in range(cv-1)]
pos_index_list.append(pos_index[(cv-1)*s:])
assert(np.sum([len(e) for e in pos_index_list])==len(pos_index))# Check on positive index completion

"""Initializing the list of outputs, predictions and probabilities to computed CV-ly"""
Y = ys
Y_Pred = []
Y_Proba = []

"""Intializing the list of cross-validated features"""
precision_list = []
recall_list = []
roc_auc_list = []
f1_list = []

"""Storing coefficients and biases for stability evaluation"""
biases = []
weights = []

"""Performing the cross-validation of the model using the features of interest
A caveat to be mentioned is that the feature selection was performed on the whole dataset, which may be a little biased towards choosing the right features.
This effect will be neglected during this test."""
for k in range(cv):
    """Splitting the data into train and test"""
    print('{}-th fold'.format(k+1))
    print('Splitting the data')
    neg_test = [X_neg[i] for i in neg_index_list[k]]
    pos_test = [X_neg[i] for i in pos_index_list[k]]
    neg_train = [X_neg[i] for i in set(neg_index).difference(neg_index_list[k])]
    pos_train = [X_neg[i] for i in set(pos_index).difference(pos_index_list[k])]

    X_train = pos_train + neg_train
    X_test = pos_test + neg_test

    y_train = [1] * len(pos_train) + [0] * len(neg_train)
    y_test = [1] * len(pos_test) + [0] * len(neg_test)

    """Fitting the model"""
    print('Fitting the model')
    d = X_train[0].shape[1]
    model = MaxLog(d=d, fit_intercept=True, alpha=10., epsilon=1e-4, init_b=logit(np.mean(y_train)), init_w=np.zeros(d))
    model.fit(X_train, y_train, n_iter=3000, parallel=False)

    print('Evaluation and storage of model parameters\n')
    """Outputting the predictions and the probability scores"""
    y_score = model.predict_proba(X_test)
    y_pred = model.predict(X_test)

    """Computing the various metrics"""
    f1_list.append(f1_score(y_test, y_pred))
    precision_list.append(precision_score(y_test, y_pred))
    recall_list.append(recall_score(y_test, y_pred))
    roc_auc_list.append(roc_auc_score(y_test, y_score))

    """Adding predictions and scores to computed global cross-validated performance after the end of the process"""
    Y_Pred.append(list(y_pred))
    Y_Proba.append(list(y_score))

    """Storing the model's coefficients"""
    biases.append(model.b)
    weights.append(model.w)

print("Cross-validation evaluation of the NeighLoR approach:")
print("Precision : {} +- {}\nRecall : {} +- {}\nF1 : {} +- {}\nROC AUC : {} +- {}".format(np.mean(precision_list), np.std(precision_list),
                                                                                          np.mean(recall_list), np.std(recall_list),
                                                                                          np.mean(f1_list), np.std(f1_list),
                                                                                          np.mean(roc_auc_list), np.std(roc_auc_list)))

1-th fold
Splitting the data
Fitting the model



Evaluation and storage of model parameters

2-th fold
Splitting the data
Fitting the model



Evaluation and storage of model parameters

3-th fold
Splitting the data
Fitting the model



Evaluation and storage of model parameters

4-th fold
Splitting the data
Fitting the model



Evaluation and storage of model parameters

5-th fold
Splitting the data
Fitting the model


...


Evaluation and storage of model parameters

7-th fold
Splitting the data
Fitting the model



Evaluation and storage of model parameters

8-th fold
Splitting the data
Fitting the model



Evaluation and storage of model parameters

9-th fold
Splitting the data
Fitting the model



Evaluation and storage of model parameters

10-th fold
Splitting the data
Fitting the model



Evaluation and storage of model parameters

Precision : 0.864574833040054 +- 0.029080479987617792
Recall : 0.7517989629038692 +- 0.04295139434114574
F1 : 0.8033715953618777 +- 0.027473547512926404
ROC AUC : 0.9654022805806847 +- 0.01200859868767601


In [71]:
"""Training the model used to evaluate the out-of-sample data"""
ys = list(data.pi)
d = X_neg[0].shape[1]
model = MaxLog(d=d, fit_intercept=True, alpha=10., epsilon=1e-4, init_b=logit(np.mean(ys)), init_w=np.zeros(d))
model.fit(X_neg, ys, n_iter=5000)
pickle.dump((model, feat), open("./BA-final-model.b","wb"), protocol=pickle.HIGHEST_PROTOCOL)

In [72]:
model, feat = pickle.load(open("./BA-final-model.b",'rb'))
coef_ = pd.Series({feat[k]: model.w[k] for k in range(len(feat))}).sort_values(ascending=False)

In [73]:
coef_

boycott          7.365126
again            4.346433
avoid            4.117051
hell             4.073102
never            3.841027
virgin           3.688061
rip              3.606456
future           3.372269
fly              3.185931
racist           2.794640
consider         2.760598
not_choose       2.672799
unacceptable     2.642037
bother           2.586675
trust            2.584697
extremely        2.569666
not_buy          2.520290
terrible         2.485585
recommend        2.417858
short            2.400888
total            2.358891
use              2.350799
refuse           2.322920
glad             2.213056
book             2.091789
highly           2.068966
horrible         1.988287
not_recommend    1.957959
wonder           1.910302
disgusting       1.871725
                   ...   
now             -1.806789
lose            -1.817357
as              -1.827593
website         -1.854154
cancel          -1.973457
there           -2.012327
any             -2.060053
ship        

## Evaluating logistic regression

In [38]:
"""Building global one-hot encoding representation"""
feat_reps_1d = np.array([[int(w in fr) for w in features_20] for fr in tqdm(full_representation)])
neg_reps_1d = np.array([[int(v in nr) for v in verb_bigrams] for nr in tqdm(neg_reps)])
X_neg_1d = np.hstack([feat_reps_1d, neg_reps_1d, np.array(from_ba).reshape(-1,1)])

/home/qrg-researchlab/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  This is separate from the ipykernel package so we can avoid doing imports until


In [42]:
"""Building indexes"""
n = len(ys)

"""Retrieving the positive and negative indexes"""
neg_index = [k for k in range(len(ys)) if ys[k]==0]
pos_index = [k for k in range(len(ys)) if ys[k]==1]

"""Shuffling both the positive and negative indexes"""
np.random.seed(seed=0)
np.random.shuffle(neg_index)
np.random.shuffle(pos_index)

"""Computing train and test index sets for the chosen number of folds"""
cv = 10

s = len(neg_index)//cv
neg_index_list = [neg_index[k*s:(k+1)*s] for k in range(cv-1)]
neg_index_list.append(neg_index[(cv-1)*s:])
assert(np.sum([len(e) for e in neg_index_list])==len(neg_index)) # Check on negative index completion

s = len(pos_index)//cv
pos_index_list = [pos_index[k*s:(k+1)*s] for k in range(cv-1)]
pos_index_list.append(pos_index[(cv-1)*s:])
assert(np.sum([len(e) for e in pos_index_list])==len(pos_index))# Check on positive index completion

"""Initializing the list of outputs, predictions and probabilities to computed CV-ly"""
Y = ys
Y_Pred = []
Y_Proba = []

"""Intializing the list of cross-validated features"""
precision_list = []
recall_list = []
roc_auc_list = []
f1_list = []

"""Storing coefficients and biases for stability evaluation"""
biases = []
weights = []

"""Performing the cross-validation of the model using the features of interest
A caveat to be mentioned is that the feature selection was performed on the whole dataset, which may be a little biased towards choosing the right features.
This effect will be neglected during this test."""
for k in range(cv):
    """Splitting the data into train and test"""
    print('{}-th fold'.format(k+1))
    print('Splitting the data')
    neg_test = [X_neg_1d[i:i+1,:] for i in neg_index_list[k]]
    pos_test = [X_neg_1d[i:i+1,:] for i in pos_index_list[k]]
    neg_train = [X_neg_1d[i:i+1,:] for i in set(neg_index).difference(neg_index_list[k])]
    pos_train = [X_neg_1d[i:i+1,:] for i in set(pos_index).difference(pos_index_list[k])]

    X_train = np.vstack(pos_train + neg_train)
    X_test = np.vstack(pos_test + neg_test)

    y_train = [1] * len(pos_train) + [0] * len(neg_train)
    y_test = [1] * len(pos_test) + [0] * len(neg_test)

    """Fitting the model"""
    print('Fitting the model')
    model = LogisticRegression(C=1e4)
    model.fit(X_train, y_train)

    print('Evaluation and storage of model parameters\n')
    """Outputting the predictions and the probability scores"""
    y_score = model.predict_proba(X_test)[:,1]
    y_pred = model.predict(X_test)

    """Computing the various metrics"""
    f1_list.append(f1_score(y_test, y_pred))
    precision_list.append(precision_score(y_test, y_pred))
    recall_list.append(recall_score(y_test, y_pred))
    roc_auc_list.append(roc_auc_score(y_test, y_score))

    """Adding predictions and scores to computed global cross-validated performance after the end of the process"""
    Y_Pred.append(list(y_pred))
    Y_Proba.append(list(y_score))

print("Cross-validation evaluation for the logistic regression:")
print("Precision : {} +- {}\nRecall : {} +- {}\nF1 : {} +- {}\nROC AUC : {} +- {}".format(np.mean(precision_list), np.std(precision_list),
                                                                                          np.mean(recall_list), np.std(recall_list),
                                                                                          np.mean(f1_list), np.std(f1_list),
                                                                                          np.mean(roc_auc_list), np.std(roc_auc_list)))

1-th fold
Splitting the data
Fitting the model
Evaluation and storage of model parameters

2-th fold
Splitting the data
Fitting the model
Evaluation and storage of model parameters

3-th fold
Splitting the data
Fitting the model
Evaluation and storage of model parameters

4-th fold
Splitting the data
Fitting the model
Evaluation and storage of model parameters

5-th fold
Splitting the data
Fitting the model
Evaluation and storage of model parameters

6-th fold
Splitting the data
Fitting the model
Evaluation and storage of model parameters

7-th fold
Splitting the data
Fitting the model
Evaluation and storage of model parameters

8-th fold
Splitting the data
Fitting the model
Evaluation and storage of model parameters

9-th fold
Splitting the data
Fitting the model
Evaluation and storage of model parameters

10-th fold
Splitting the data
Fitting the model
Evaluation and storage of model parameters

Cross-validation evaluation for the logistic regression:
Precision : 0.7677593029044705 +

## Pretrained Google Word2Vec + Logistic regression

In [48]:
"""Loading W2V matrix and building representation"""
from gensim.models import KeyedVectors

google_w2v = KeyedVectors.load_word2vec_format('/home/qrg-researchlab/Downloads/GoogleNews-vectors-negative300.bin', binary=True)

def get_embed(x):
    local = list(x[x.tag.isin(['V','A','R','N'])].word)
    embed = [google_w2v[w].reshape((1,-1)) for w in local if w in google_w2v.vocab.keys()]
    if len(embed)>0:
        return np.vstack(embed).sum(axis=0)
    else:
        return np.zeros(300)

X_pt_w2v = np.vstack([get_embed(data.tag_df.iloc[k]).reshape((1,-1)) for k in tqdm(range(data.shape[0]))])

In [49]:
from scipy.special import expit, logit
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
from sklearn.linear_model import LogisticRegression

"""Building indexes"""
n = len(ys)
d = X_pt_w2v.shape[1]

"""Retrieving the positive and negative indexes"""
neg_index = [k for k in range(len(ys)) if ys[k]==0]
pos_index = [k for k in range(len(ys)) if ys[k]==1]

"""Shuffling both the positive and negative indexes"""
np.random.seed(seed=0)
np.random.shuffle(neg_index)
np.random.shuffle(pos_index)

"""Computing train and test index sets for the chosen number of folds"""
cv = 10

s = len(neg_index)//cv
neg_index_list = [neg_index[k*s:(k+1)*s] for k in range(cv-1)]
neg_index_list.append(neg_index[(cv-1)*s:])
assert(np.sum([len(e) for e in neg_index_list])==len(neg_index)) # Check on negative index completion

s = len(pos_index)//cv
pos_index_list = [pos_index[k*s:(k+1)*s] for k in range(cv-1)]
pos_index_list.append(pos_index[(cv-1)*s:])
assert(np.sum([len(e) for e in pos_index_list])==len(pos_index))# Check on positive index completion

"""Initializing the list of outputs, predictions and probabilities to computed CV-ly"""
Y = ys
Y_Pred = []
Y_Proba = []

"""Intializing the list of cross-validated features"""
precision_list = []
recall_list = []
roc_auc_list = []
f1_list = []

"""Performing the cross-validation of the model using the features of interest
"""
for k in range(cv):
    """Splitting the data into train and test"""
    print('{}-th fold'.format(k+1))
    print('Splitting the data')
    neg_test = np.vstack([X_pt_w2v[i:i+1] for i in neg_index_list[k]])
    pos_test = np.vstack([X_pt_w2v[i:i+1] for i in pos_index_list[k]])
    neg_train = np.vstack([X_pt_w2v[i:i+1] for i in set(neg_index).difference(neg_index_list[k])])
    pos_train = np.vstack([X_pt_w2v[i:i+1] for i in set(pos_index).difference(pos_index_list[k])])

    X_train = np.vstack([pos_train, neg_train])
    X_test = np.vstack([pos_test, neg_test])

    y_train = [1] * len(pos_train) + [0] * len(neg_train)
    y_test = [1] * len(pos_test) + [0] * len(neg_test)

    """Fitting the model"""
    print('Fitting the model')
    d = X_train.shape[1]
    model = LogisticRegression(fit_intercept=True, C=1e4)
    model.fit(X_train, y_train)

    print('Evaluation and storage of model parameters\n')
    """Outputting the predictions and the probability scores"""
    y_score = model.predict_proba(X_test)[:,1]
    y_pred = model.predict(X_test)

    """Computing the various metrics"""
    f1_list.append(f1_score(y_test, y_pred))
    precision_list.append(precision_score(y_test, y_pred))
    recall_list.append(recall_score(y_test, y_pred))
    roc_auc_list.append(roc_auc_score(y_test, y_score))

    """Adding predictions and scores to computed global cross-validated performance after the end of the process"""
    Y_Pred.append(list(y_pred))
    Y_Proba.append(list(y_score))

print("Cross-validation evaluation of logistic regression + pretrained Word2Vec:")
print("Precision : {} +- {}\nRecall : {} +- {}\nF1 : {} +- {}\nROC AUC : {} +- {}".format(np.mean(precision_list), np.std(precision_list),
                                                                                          np.mean(recall_list), np.std(recall_list),
                                                                                          np.mean(f1_list), np.std(f1_list),
                                                                                          np.mean(roc_auc_list), np.std(roc_auc_list)))

1-th fold
Splitting the data
Fitting the model
Evaluation and storage of model parameters

2-th fold
Splitting the data
Fitting the model
Evaluation and storage of model parameters

3-th fold
Splitting the data
Fitting the model
Evaluation and storage of model parameters

4-th fold
Splitting the data
Fitting the model
Evaluation and storage of model parameters

5-th fold
Splitting the data
Fitting the model
Evaluation and storage of model parameters

6-th fold
Splitting the data
Fitting the model
Evaluation and storage of model parameters

7-th fold
Splitting the data
Fitting the model
Evaluation and storage of model parameters

8-th fold
Splitting the data
Fitting the model
Evaluation and storage of model parameters

9-th fold
Splitting the data
Fitting the model
Evaluation and storage of model parameters

10-th fold
Splitting the data
Fitting the model
Evaluation and storage of model parameters

Cross-validation evaluation of logistic regression + pretrained Word2Vec:
Precision : 0.7

## Pretrained Word2Vec + NeighLoR

In [53]:
"""Building representations for word embeddings"""
d = 300

def represent(x):
    rep = []
    for w in x:
        try:
            rep.append(google_w2v[w])
        except:
            pass
    if len(rep)>0:
        return np.vstack(rep).sum(axis=0)
    else:
        return np.zeros(300)

X_basic = [[represent(pattern).reshape((1,-1)) for pattern in tweet] if len(tweet)>0 else np.zeros((1,300)) for tweet in tqdm(actual_words_7)]
X_basic = [np.vstack(tweet) for tweet in tqdm(X_basic)]

for k in tqdm(range(len(X_basic))):
    if X_basic[k].shape[0]==0:
        X_basic[k] = np.zeros((1,d))

In [55]:
"""Building indexes"""
n = len(ys)
d = X_basic[0].shape[1]

"""Retrieving the positive and negative indexes"""
neg_index = [k for k in range(len(ys)) if ys[k]==0]
pos_index = [k for k in range(len(ys)) if ys[k]==1]

"""Shuffling both the positive and negative indexes"""
np.random.seed(seed=0)
np.random.shuffle(neg_index)
np.random.shuffle(pos_index)

"""Computing train and test index sets for the chosen number of folds"""
cv = 10

s = len(neg_index)//cv
neg_index_list = [neg_index[k*s:(k+1)*s] for k in range(cv-1)]
neg_index_list.append(neg_index[(cv-1)*s:])
assert(np.sum([len(e) for e in neg_index_list])==len(neg_index)) # Check on negative index completion

s = len(pos_index)//cv
pos_index_list = [pos_index[k*s:(k+1)*s] for k in range(cv-1)]
pos_index_list.append(pos_index[(cv-1)*s:])
assert(np.sum([len(e) for e in pos_index_list])==len(pos_index))# Check on positive index completion

"""Cross-validation on 10% features, without hashtags"""
from scipy.special import logit

"""Initializing the list of outputs, predictions and probabilities to computed CV-ly"""
Y = ys
Y_Pred = []
Y_Proba = []

"""Intializing the list of cross-validated features"""
precision_list = []
recall_list = []
roc_auc_list = []
f1_list = []

"""Storing coefficients and biases for stability evaluation"""
biases = []
weights = []

"""Performing the cross-validation of the model using the features of interest
A caveat to be mentioned is that the feature selection was performed on the whole dataset, which may be a little biased towards choosing the right features.
This effect will be neglected during this test."""
for k in range(cv):
    """Splitting the data into train and test"""
    print('{}-th fold'.format(k+1))
    print('Splitting the data')
    neg_test = [X_basic[i] for i in neg_index_list[k]]
    pos_test = [X_basic[i] for i in pos_index_list[k]]
    neg_train = [X_basic[i] for i in set(neg_index).difference(neg_index_list[k])]
    pos_train = [X_basic[i] for i in set(pos_index).difference(pos_index_list[k])]

    X_train = pos_train + neg_train
    X_test = pos_test + neg_test

    y_train = [1] * len(pos_train) + [0] * len(neg_train)
    y_test = [1] * len(pos_test) + [0] * len(neg_test)

    """Fitting the model"""
    print('Fitting the model')
    d = X_train[0].shape[1]
    model = MaxLog(d=d, fit_intercept=True, alpha=10., epsilon=1e-4, init_b=logit(np.mean(y_train)), init_w=np.zeros(d))
    model.fit(X_train, y_train, n_iter=3000, parallel=False)

    print('Evaluation and storage of model parameters\n')
    """Outputting the predictions and the probability scores"""
    y_score = model.predict_proba(X_test)
    y_pred = model.predict(X_test)

    """Computing the various metrics"""
    f1_list.append(f1_score(y_test, y_pred))
    precision_list.append(precision_score(y_test, y_pred))
    recall_list.append(recall_score(y_test, y_pred))
    roc_auc_list.append(roc_auc_score(y_test, y_score))

    """Adding predictions and scores to computed global cross-validated performance after the end of the process"""
    Y_Pred.append(list(y_pred))
    Y_Proba.append(list(y_score))

    """Storing the model's coefficients"""
    biases.append(model.b)
    weights.append(model.w)

print("Cross-validation evaluation of the pretrained Word2Vec + NeighLoR approach:")
print("Precision : {} +- {}\nRecall : {} +- {}\nF1 : {} +- {}\nROC AUC : {} +- {}".format(np.mean(precision_list), np.std(precision_list),
                                                                                          np.mean(recall_list), np.std(recall_list),
                                                                                          np.mean(f1_list), np.std(f1_list),
                                                                                          np.mean(roc_auc_list), np.std(roc_auc_list)))

1-th fold
Splitting the data
Fitting the model


/home/qrg-researchlab/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:114: RuntimeWarning: divide by zero encountered in log
/home/qrg-researchlab/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:114: RuntimeWarning: invalid value encountered in multiply


Evaluation and storage of model parameters

2-th fold
Splitting the data
Fitting the model


Evaluation and storage of model parameters

3-th fold
Splitting the data
Fitting the model


Evaluation and storage of model parameters

4-th fold
Splitting the data
Fitting the model


...

Evaluation and storage of model parameters

7-th fold
Splitting the data
Fitting the model


Evaluation and storage of model parameters

8-th fold
Splitting the data
Fitting the model


Evaluation and storage of model parameters

9-th fold
Splitting the data
Fitting the model


Evaluation and storage of model parameters

10-th fold
Splitting the data
Fitting the model


Evaluation and storage of model parameters

Cross-validation evaluation of the pretrained Word2Vec + NeighLoR approach:
Precision : 0.7711294427745828 +- 0.15406778609165464
Recall : 0.6301874750698047 +- 0.30800864781456283
F1 : 0.603518495723111 +- 0.22403213065299948
ROC AUC : 0.9373578827964302 +- 0.038660013566189505


## Training custom Word2Vec

In [61]:
"""Loading large BA data to train our own specialized embedding"""
from gensim.models import Word2Vec
import os
from nltk import WordNetLemmatizer

"""Loading British Airways data"""
big_data = []

for file in tqdm(os.listdir('../processed_data/british%20airways/')):
    if file[-2:]=='.b':
        big_data.append(pickle.load(open(os.path.join('../processed_data/british%20airways/', file), 'rb')))

data_df = pd.concat(big_data, axis=0, sort=False)
data_df.index = list(range(data_df.shape[0]))

"""Collecting verbs, adverbs, and adjectives"""
lmtzr = WordNetLemmatizer()
unique_verbs = set()
unique_adverbs = set()
unique_adjectives = set()

for k in tqdm(range(data_df.shape[0])):
    local = data_df.tag_df.iloc[k]
    try:
        local = local[local.word.apply(lambda x: x.count('-')<=1 and x.count('/')<=1 and not(any(str(k) in x for k in range(10))) and not('#' in x))]
        unique_verbs.update(list(local[local.tag=='V'].word.apply(lambda x: lmtzr.lemmatize(x.replace('-','').replace('/','').replace('\\',''),'v'))))
        unique_adverbs.update(list(local[local.tag=='R'].word.apply(lambda x: x.replace('-','').replace('/','').replace('\\',''))))
        unique_adjectives.update(list(local[local.tag=='A'].word.apply(lambda x: x.replace('-','').replace('/','').replace('\\',''))))
    except:
        pass
    
"""Lemmatizing all verbs"""
lem_verbs = {verb: lmtzr.lemmatize(verb, 'v') for verb in unique_verbs}
lem_adj = {adj: lmtzr.lemmatize(adj, 'a') for adj in unique_adjectives}
lem_adv = {adv: adv for adv in unique_adverbs}
lem_words = dict()
lem_words.update(lem_adv)
lem_words.update(lem_adj)
lem_words.update(lem_verbs)

/home/qrg-researchlab/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


AttributeError: 'float' object has no attribute 'word'

In [63]:
"""Applying lemmatization to the text"""
to_remove = []
for k in tqdm(range(data_df.shape[0])):
    local = data_df.tag_df.iloc[k]
    try:
        local['word'] = local.word.apply(lambda x: lem_words[x] if x in lem_words.keys() else x)
        data_df.tag_df['word'] = deepcopy(local.word)
    except:
        to_remove.append(k)

"""Training the embedding"""
sentences = [list(data_df.tag_df.iloc[k][data_df.tag_df.iloc[k].tag.isin(['V','R','A','N'])].word) for k in tqdm(set(list(range(data_df.shape[0]))).difference(to_remove))]
w2v_model = Word2Vec(min_count=20,
                     window=2,
                     size=50,
                     alpha=0.05,
                     min_alpha=0.001,
                     negative=5,
                     workers=40)
w2v_model.build_vocab(sentences, progress_per=10000)
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

(107018275, 129195600)

## Custom Word2Vec + logistic regression

In [64]:
"""Building the representation"""
def get_embed(x):
    local = list(x[x.tag.isin(['V','A','R','N'])].word)
    embed = [w2v_model[w].reshape((1,-1)) for w in local if w in w2v_model.wv.vocab.keys()]
    if len(embed)>0:
        return np.vstack(embed).sum(axis=0)
    else:
        return np.zeros(50)

X_pt_w2v = np.vstack([get_embed(data.tag_df.iloc[k]).reshape((1,-1)) for k in tqdm(range(data.shape[0]))])

/home/qrg-researchlab/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.


In [65]:
"""Building indexes"""
n = len(ys)
d = X_pt_w2v.shape[1]

"""Retrieving the positive and negative indexes"""
neg_index = [k for k in range(len(ys)) if ys[k]==0]
pos_index = [k for k in range(len(ys)) if ys[k]==1]

"""Shuffling both the positive and negative indexes"""
np.random.seed(seed=0)
np.random.shuffle(neg_index)
np.random.shuffle(pos_index)

"""Computing train and test index sets for the chosen number of folds"""
cv = 10

s = len(neg_index)//cv
neg_index_list = [neg_index[k*s:(k+1)*s] for k in range(cv-1)]
neg_index_list.append(neg_index[(cv-1)*s:])
assert(np.sum([len(e) for e in neg_index_list])==len(neg_index)) # Check on negative index completion

s = len(pos_index)//cv
pos_index_list = [pos_index[k*s:(k+1)*s] for k in range(cv-1)]
pos_index_list.append(pos_index[(cv-1)*s:])
assert(np.sum([len(e) for e in pos_index_list])==len(pos_index))# Check on positive index completion

"""Cross-validation on 10% features, without hashtags"""
from scipy.special import logit

"""Initializing the list of outputs, predictions and probabilities to computed CV-ly"""
Y = ys
Y_Pred = []
Y_Proba = []

"""Intializing the list of cross-validated features"""
precision_list = []
recall_list = []
roc_auc_list = []
f1_list = []

"""Performing the cross-validation of the model using the features of interest
A caveat to be mentioned is that the feature selection was performed on the whole dataset, which may be a little biased towards choosing the right features.
This effect will be neglected during this test."""
for k in range(cv):
    """Splitting the data into train and test"""
    print('{}-th fold'.format(k+1))
    print('Splitting the data')
    neg_test = np.vstack([X_pt_w2v[i:i+1] for i in neg_index_list[k]])
    pos_test = np.vstack([X_pt_w2v[i:i+1] for i in pos_index_list[k]])
    neg_train = np.vstack([X_pt_w2v[i:i+1] for i in set(neg_index).difference(neg_index_list[k])])
    pos_train = np.vstack([X_pt_w2v[i:i+1] for i in set(pos_index).difference(pos_index_list[k])])

    X_train = np.vstack([pos_train, neg_train])
    X_test = np.vstack([pos_test, neg_test])

    y_train = [1] * len(pos_train) + [0] * len(neg_train)
    y_test = [1] * len(pos_test) + [0] * len(neg_test)

    """Fitting the model"""
    print('Fitting the model')
    d = X_train.shape[1]
    model = LogisticRegression(fit_intercept=True, C=1e4)
    model.fit(X_train, y_train)

    print('Evaluation and storage of model parameters\n')
    """Outputting the predictions and the probability scores"""
    y_score = model.predict_proba(X_test)[:,1]
    y_pred = model.predict(X_test)

    """Computing the various metrics"""
    f1_list.append(f1_score(y_test, y_pred))
    precision_list.append(precision_score(y_test, y_pred))
    recall_list.append(recall_score(y_test, y_pred))
    roc_auc_list.append(roc_auc_score(y_test, y_score))

    """Adding predictions and scores to computed global cross-validated performance after the end of the process"""
    Y_Pred.append(list(y_pred))
    Y_Proba.append(list(y_score))

print("Cross-validation evaluation of custom Word2Vec + logistic regression:")
print("Precision : {} +- {}\nRecall : {} +- {}\nF1 : {} +- {}\nROC AUC : {} +- {}".format(np.mean(precision_list), np.std(precision_list),
                                                                                          np.mean(recall_list), np.std(recall_list),
                                                                                          np.mean(f1_list), np.std(f1_list),
                                                                                          np.mean(roc_auc_list), np.std(roc_auc_list)))

1-th fold
Splitting the data
Fitting the model
Evaluation and storage of model parameters

2-th fold
Splitting the data
Fitting the model
Evaluation and storage of model parameters

3-th fold
Splitting the data
Fitting the model
Evaluation and storage of model parameters

4-th fold
Splitting the data
Fitting the model
Evaluation and storage of model parameters

5-th fold
Splitting the data
Fitting the model
Evaluation and storage of model parameters

6-th fold
Splitting the data
Fitting the model
Evaluation and storage of model parameters

7-th fold
Splitting the data
Fitting the model
Evaluation and storage of model parameters

8-th fold
Splitting the data
Fitting the model
Evaluation and storage of model parameters

9-th fold
Splitting the data
Fitting the model
Evaluation and storage of model parameters

10-th fold
Splitting the data
Fitting the model
Evaluation and storage of model parameters

Cross-validation evaluation of custom Word2Vec + logistic regression:
Precision : 0.71882

## Custom Word2Vec + NeighLoR

In [66]:
"""Building representations for word embeddings"""
d = 50

def represent(x):
    rep = []
    for w in x:
        try:
            rep.append(w2v_model[w])
        except:
            pass
    if len(rep)>0:
        return np.vstack(rep).sum(axis=0)
    else:
        return np.zeros(d)

X_basic = [[represent(pattern).reshape((1,-1)) for pattern in tweet] if len(tweet)>0 else np.zeros((1,d)) for tweet in tqdm(actual_words_7)]
X_basic = [np.vstack(X_basic[k]) for k in tqdm(range(len(X_basic)))]


for k in tqdm(range(len(X_basic))):
    if X_basic[k].shape[0]==0:
        X_basic[k] = np.zeros((1,d))

/home/qrg-researchlab/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [67]:
from scipy.special import expit, logit
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

"""Building indexes"""
n = len(ys)
d = X_basic[0].shape[1]

"""Retrieving the positive and negative indexes"""
neg_index = [k for k in range(len(ys)) if ys[k]==0]
pos_index = [k for k in range(len(ys)) if ys[k]==1]

"""Shuffling both the positive and negative indexes"""
np.random.seed(seed=0)
np.random.shuffle(neg_index)
np.random.shuffle(pos_index)

"""Computing train and test index sets for the chosen number of folds"""
cv = 10

s = len(neg_index)//cv
neg_index_list = [neg_index[k*s:(k+1)*s] for k in range(cv-1)]
neg_index_list.append(neg_index[(cv-1)*s:])
assert(np.sum([len(e) for e in neg_index_list])==len(neg_index)) # Check on negative index completion

s = len(pos_index)//cv
pos_index_list = [pos_index[k*s:(k+1)*s] for k in range(cv-1)]
pos_index_list.append(pos_index[(cv-1)*s:])
assert(np.sum([len(e) for e in pos_index_list])==len(pos_index))# Check on positive index completion

"""Cross-validation on 10% features, without hashtags"""
from scipy.special import logit

"""Initializing the list of outputs, predictions and probabilities to computed CV-ly"""
Y = ys
Y_Pred = []
Y_Proba = []

"""Intializing the list of cross-validated features"""
precision_list = []
recall_list = []
roc_auc_list = []
f1_list = []

"""Storing coefficients and biases for stability evaluation"""
biases = []
weights = []

"""Performing the cross-validation of the model using the features of interest
A caveat to be mentioned is that the feature selection was performed on the whole dataset, which may be a little biased towards choosing the right features.
This effect will be neglected during this test."""
for k in range(cv):
    """Splitting the data into train and test"""
    print('{}-th fold'.format(k+1))
    print('Splitting the data')
    neg_test = [X_basic[i] for i in neg_index_list[k]]
    pos_test = [X_basic[i] for i in pos_index_list[k]]
    neg_train = [X_basic[i] for i in set(neg_index).difference(neg_index_list[k])]
    pos_train = [X_basic[i] for i in set(pos_index).difference(pos_index_list[k])]

    X_train = pos_train + neg_train
    X_test = pos_test + neg_test

    y_train = [1] * len(pos_train) + [0] * len(neg_train)
    y_test = [1] * len(pos_test) + [0] * len(neg_test)

    """Fitting the model"""
    print('Fitting the model')
    d = X_train[0].shape[1]
    model = MaxLog(d=d, fit_intercept=True, alpha=10., epsilon=1e-4, init_b=logit(np.mean(y_train)), init_w=np.zeros(d))
    model.fit(X_train, y_train, n_iter=3000, parallel=False)

    print('Evaluation and storage of model parameters\n')
    """Outputting the predictions and the probability scores"""
    y_score = model.predict_proba(X_test)
    y_pred = model.predict(X_test)

    """Computing the various metrics"""
    f1_list.append(f1_score(y_test, y_pred))
    precision_list.append(precision_score(y_test, y_pred))
    recall_list.append(recall_score(y_test, y_pred))
    roc_auc_list.append(roc_auc_score(y_test, y_score))

    """Adding predictions and scores to computed global cross-validated performance after the end of the process"""
    Y_Pred.append(list(y_pred))
    Y_Proba.append(list(y_score))

    """Storing the model's coefficients"""
    biases.append(model.b)
    weights.append(model.w)
    
print("Precision : {} +- {}\nRecall : {} +- {}\nF1 : {} +- {}\nROC AUC : {} +- {}".format(np.mean(precision_list), np.std(precision_list),
                                                                                          np.mean(recall_list), np.std(recall_list),
                                                                                          np.mean(f1_list), np.std(f1_list),
                                                                                          np.mean(roc_auc_list), np.std(roc_auc_list)))

1-th fold
Splitting the data
Fitting the model


/home/qrg-researchlab/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:114: RuntimeWarning: divide by zero encountered in log
/home/qrg-researchlab/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:114: RuntimeWarning: invalid value encountered in multiply


Evaluation and storage of model parameters

2-th fold
Splitting the data
Fitting the model


Evaluation and storage of model parameters

3-th fold
Splitting the data
Fitting the model


Evaluation and storage of model parameters

4-th fold
Splitting the data
Fitting the model


...

Evaluation and storage of model parameters

7-th fold
Splitting the data
Fitting the model


Evaluation and storage of model parameters

8-th fold
Splitting the data
Fitting the model


Evaluation and storage of model parameters

9-th fold
Splitting the data
Fitting the model


Evaluation and storage of model parameters

10-th fold
Splitting the data
Fitting the model


Evaluation and storage of model parameters

Precision : 0.5005134923924911 +- 0.17389033353749067
Recall : 0.3455684084563223 +- 0.26311095458875655
F1 : 0.323154679297601 +- 0.1735324810461968
ROC AUC : 0.7807655438904357 +- 0.06943205690523833
